In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
%cd "/content/drive/My Drive/data/audio"

/content/drive/My Drive/data/audio


In [0]:
!pip install librosa
!pip install xlrd

    100% |████████████████████████████████| 1.6MB 13.9MB/s 
    100% |████████████████████████████████| 327kB 22.6MB/s 
    100% |████████████████████████████████| 3.2MB 10.7MB/s 
    100% |████████████████████████████████| 16.1MB 2.8MB/s 
  Running setup.py bdist_wheel for librosa ... - \ done
  Stored in directory: /root/.cache/pip/wheels/18/b8/10/f0f8f6ac60668a5cd75596cf14c25bb6b3ea1ecd815f058b7e
  Running setup.py bdist_wheel for audioread ... - done
  Stored in directory: /root/.cache/pip/wheels/53/02/90/7b5c4081b7470c550ab605f600bad237dde12a6b8999b11f50
  Running setup.py bdist_wheel for resampy ... - \ done
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built librosa audioread resampy


In [0]:
import librosa
import glob
import pandas as pd
import numpy as np
import keras
import xlrd
import re
import cv2
import random
from sklearn.preprocessing import LabelEncoder



In [0]:
#extract song names
songs = [re.search('.+?(?=\.wav)', x).group(0)
              for x in glob.iglob('*.wav')]

#extract song meter
meter = []
for song in songs:
  with open('../annotations/meter/' + song + '.meter', 'r') as file:
      meter.append(file.readline())

In [0]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        z = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            y, sr = librosa.load(ID + '.wav', offset = 30, duration = 2.97)
            ps = librosa.feature.melspectrogram(y=y, sr=sr)
            X[i,] = cv2.merge((ps,ps,ps))
            # Store class
            z[i] = self.labels[ID]

        return X, keras.utils.to_categorical(z, num_classes=self.n_classes)

In [0]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import numpy as np
import glob
import re
import math
import keras
from keras import regularizers
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from random import shuffle
from keras.datasets import mnist

Using TensorFlow backend.


In [0]:
from keras.layers import Input
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K


# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 4 classes
predictions = Dense(4, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


for layer in model.layers:
   layer.trainable = True

# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
              loss='categorical_crossentropy',  metrics=['accuracy'])
       
# Parameters
params = {'dim': (128, 128),
          'batch_size': 5,
          'n_classes': 4,
          'n_channels': 3,
          'shuffle': True}
frac = 0.7

# Datasets
le = LabelEncoder()
meter = le.fit_transform(meter) #encode meter to integers
song_meter = list(zip(songs, meter))
random.shuffle(song_meter)

songs = [x[0] for x in song_meter]
split_index = int(np.floor(len(songs) * frac))


partition = {'train': songs[:split_index], 'validation': songs[split_index:]}
labels = dict(song_meter)

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

# train the model on the new data for a few epochs
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator)



Train on 80 samples, validate on 38 samples
Epoch 1/200
80/80 [==============================] - 34s 426ms/step - loss: 1.4603 - acc: 0.2625 - val_loss: 1.4682 - val_acc: 0.1842
Epoch 2/200
80/80 [==============================] - 4s 53ms/step - loss: 1.4050 - acc: 0.2625 - val_loss: 1.5100 - val_acc: 0.2368
Epoch 3/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3517 - acc: 0.3500 - val_loss: 1.4930 - val_acc: 0.2368
Epoch 4/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3640 - acc: 0.3125 - val_loss: 1.4737 - val_acc: 0.2368
Epoch 5/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3251 - acc: 0.3750 - val_loss: 1.4954 - val_acc: 0.1316
Epoch 6/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3484 - acc: 0.2625 - val_loss: 1.4711 - val_acc: 0.2105
Epoch 7/200
80/80 [==============================] - 4s 53ms/step - loss: 1.3009 - acc: 0.4000 - val_loss: 1.4967 - val_acc: 0.2368
Epoch 8/200
80/80 [===========